<h1>Explorando el Interior de Modelos de Lenguaje Transformer</h1>
<i>Un análisis profundo de la arquitectura transformer en modelos generativos de lenguaje</i>

---

Este cuaderno está adaptado del Capítulo 3 del libro [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961)

### [OPCIONAL] - Instalación de Paquetes en <img src="https://colab.google/static/images/icons/colab.png" width=100>

Si estás ejecutando este cuaderno en Google Colab, necesitas **descomentar y ejecutar** el siguiente bloque de código para instalar las dependencias:

---

💡 **NOTA**: Para mejores resultados, usa una GPU. En Google Colab, ve a:
**Entorno de ejecución > Cambiar tipo de entorno de ejecución > Acelerador de hardware > GPU > Tipo de GPU > T4**.

---

In [1]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

# Cargando el Modelo de Lenguaje

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Cargar modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

# Crear un pipeline de generación de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch # Import torch to check for GPU availability if needed

# Cargar modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Check if CUDA is available and set device_map accordingly
# If CUDA is not available, the model will be loaded on the CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map=device, # Use the determined device
    torch_dtype="auto",
    trust_remote_code=False,
)

# Crear un pipeline de generación de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
    # Remove the device argument here, as device_map in from_pretrained handles device placement
    # device=device
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


# Entradas y Salidas de un Modelo Transformer Entrenado

In [5]:
# Ejemplo adaptado al contexto argentino
prompt = "Escribe un mensaje disculpándote con Juan por haber pisado su pie en el colectivo. Explica cómo sucedió."

output = generator(prompt)

print(output[0]['generated_text'])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.





Solución 1:

Estimado Juan,


Quisiera expresarle mi más sincera disculpa por el incidente que ocurrió en el colectivo esta mañana.


In [6]:
# Mostrar la arquitectura del modelo
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=32064, 

# Seleccionando un token de la distribución de probabilidades (sampling/decoding)

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch # Importar torch para verificar la disponibilidad de la GPU

# Cargar modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Verificar si CUDA está disponible y establecer el dispositivo en consecuencia
# Si CUDA no está disponible, el modelo se cargará en la CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}") # Imprimir el dispositivo que se está utilizando

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map=device, # Usar el dispositivo determinado
    torch_dtype="auto",
    trust_remote_code=False,
)

# Crear un pipeline de generación de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
    # Puedes pasar el dispositivo al pipeline también si es necesario,
    # pero device_map en from_pretrained generalmente maneja esto.
    # device=device
)

Usando dispositivo: cpu


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [11]:
prompt = "La capital de Argentina es"

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Mover el tensor al dispositivo determinado (cuda o cpu)
input_ids = input_ids.to(device)

# Obtener la salida del modelo antes de la capa lm_head
model_output = model.model(input_ids)

# Obtener la salida de la capa lm_head
lm_head_output = model.lm_head(model_output[0])

In [ ]:
prompt = "La capital de Argentina es"

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

# Obtener la salida del modelo antes de la capa lm_head
model_output = model.model(input_ids)

# Obtener la salida de la capa lm_head
lm_head_output = model.lm_head(model_output[0])

In [19]:
# Obtener el token con mayor probabilidad
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'\n'

# Acelerando la generación con caché de keys y values

In [14]:
# Obtener la salida del modelo antes de la capa lm_head
model_output = model.model(input_ids)

# Obtener la salida de la capa lm_head
lm_head_output = model.lm_head(model_output[0])

In [15]:
prompt = "Escribe una carta muy larga explicando por qué el asado del domingo se quemó. Describe qué salió mal."

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# Use the 'device' variable determined by the CUDA check instead of hardcoding "cuda"
input_ids = input_ids.to(device)

In [ ]:
prompt = "Escribe una carta muy larga explicando por qué el asado del domingo se quemó. Describe qué salió mal."

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [17]:
%%timeit -n 1
# Generar texto con caché activado
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


2min 10s ± 6.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit -n 1
# Generar texto con caché desactivado
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)

KeyboardInterrupt: 

# Explicación

## ¿Qué aprendimos?

1. **Carga de modelos**: Cargamos el modelo Phi-3-mini, un modelo de lenguaje eficiente y potente.
2. **Generación de texto**: Vimos cómo el modelo puede generar texto coherente en español con ejemplos locales.
3. **Arquitectura interna**: Exploramos la estructura del modelo transformer.
4. **Proceso de decodificación**: Analizamos cómo el modelo selecciona la siguiente palabra.
5. **Optimización**: Comparamos el rendimiento con y sin caché de atención.

## Aplicaciones

Estos modelos pueden usarse para:
- Generar contenido educativo en español rioplatense
- Analizar textos legales y normativas locales
- Crear asistentes virtuales con conocimiento de cultura argentina
- Procesar documentos históricos y literarios nacionales